<a href="https://colab.research.google.com/github/t1seo/Python_Notebook/blob/master/effective_python/%EC%BB%B4%ED%94%84%EB%A6%AC%ED%97%A8%EC%85%98%EA%B3%BC%20%EC%A0%9C%EB%84%88%EB%A0%88%EC%9D%B4%ED%84%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CHAPTER 4 컴프리헨션과 제너레이터

- 파이썬에서는 **컴프리헨션(comprehension)**이라는 특별한 구문을 사용해, 리스트, 딕셔너리, 집합 등의 타입을 간결하게 이터레이션하면서 원소로부터 파생되는 데이터 구조르 생성할 수 있다.

- 컴프리헨션 코딩 스타일은 **제너레이터(generator)**를 사용하는 함수로 확장할 수 있다. 제너레이터는 함수가 점진적으로 반환하는 값으로 이뤄지는 스트림을 만들어준다. 이터레이터를 사용할 수 있는 곳(foraㅜㄴ, 별표 식)이라면 어디에서나 제너레이터 함수를 호출한 결과를 사요할 수 있다.


제너레이터를 사용하면 성능을 향상시키고, 메모리 사용을 줄이고, 가독성을 높일 수 있다.

## BETTER WAY 27 map과 filter 대신 컴프리헨션을 사용하라

## BETTER WAY 28 컴프리헨션 내부에 제어 하위 식을 세 개 이상 사용하지 말라

## BETTER WAY 29 대입식을 사용해 컴프리헨션 안에서 반복 작업을 피하라

## BETTER WAY 30 리스트를 반환하기보다는 제너레이터를 사용하라

In [2]:
# 제너레이터 사용 안한 코드
def index_words(text):
    result = []
    if text:
        result.append(0)
    for index, letter in enumerate(text):
        if letter == ' ':
            result.append(index + 1)
    return result

address = '컴퓨터(영어: Computer, 문화어: 콤퓨터, 순화어: 전산기)는 진공관'
result = index_words(address)
print(result)

[0, 8, 18, 23, 28, 33, 39]


In [3]:
# 제너레이터 사용
def index_words_iter(text):
    if text:
        yield 0
    for index, letter in enumerate(text):
        if letter == ' ':
            yield index + 1

- 이 함수가 호출되면 제너레이터 함수가 실행되지 않고 즉시 이터레이터를 반환한다.
- 이터레이터가 `next` 내장 함수를 호출할 때마다 이터레이너튼 제너레이터 함수를 다음 `yield` 식까지 진행시킨다.
- 제너레이터가 `yield`에 전달하는 값은 이터레이터에 의해 호출하는 쪽에 반환된다.

In [4]:
it = index_words_iter(address)
print(next(it))
print(next(it))

0
8


In [5]:
result = list(index_words_iter(address)) # 이터레이터를 리스트로
print(result)

[0, 8, 18, 23, 28, 33, 39]


In [6]:
# 파일에서 한 번에 한 줄씩 읽어 한 단어씩 출력하는 제너레이터
def index_file(handle):
    offset = 0
    for line in handle:
        if line:
            yield offset
        for letter in line:
            offset += 1
            if letter == ' ':
                yield offset
            

In [7]:
from google.colab import files
uploaded = files.upload() # 파일 업로드 기능 실행

Saving lorem_ipsum.txt to lorem_ipsum.txt


In [11]:
import itertools

with open('lorem_ipsum.txt', 'r', encoding='utf-8') as f:
    # lorem_ipsum = f.read()
    # print(lorem_ipsum)
    it = index_file(f)
    results = itertools.islice(it, 0, 10)
    print(list(results))

[0, 6, 12, 18, 22, 28, 40, 51, 57, 64]


## BETTER WAY 31 인자에 대해 이터레이션할 때는 방어적이 돼라
- [Python | Difference between iterable and iterator](https://www.geeksforgeeks.org/python-difference-iterable-iterator/)
- [파이썬 이터레이터(iterator)와 이터러블(iterable) 차이점](https://sikaleo.tistory.com/61)

In [5]:
# 정규화 함수
def normalize(numbers):
    total = sum(numbers) # 이터레이터 사용 1
    result = []
    for value in numbers: # 이터레이터 사용 2
        percent = 100 * value / total # 각각 x 100 / 총합 => normalization
        result.append(percent)
    return result

visits = [15, 35, 80]
percentages = normalize(visits)
print(percentages)
assert sum(percentages) == 100.0

[11.538461538461538, 26.923076923076923, 61.53846153846154]


In [6]:
from google.colab import files
uploaded = files.upload() # 파일 업로드 기능 실행

Saving numbers.txt to numbers.txt


In [8]:
# 파일을 읽는 제너레이터 함수
def read_visits(data_path):
    with open(data_path) as f:
        for line in f:
            yield int(line) # yield

it = read_visits('numbers.txt')
percentages = normalize(it) # 이터레이터 두 번 사용
print(percentages) # 아무런 결과가 출력이 안된다

[]


- 위 코드는 실행이 안된다. normalize 함수에서 `sum`과 `for`에서 이터레이터를 두 번 사용하기 때문이다.

- 이터레이터는 결과를 단 한번만 만들어낸다. 이미 `StopIteration` 예외가 발생한 이터레이터나 제너레이터를 다시 이터레이션하면 아무런 결과도 얻을 수 없다.




해결 방법:
- 이터레이터를 명시적으로 소진시키고 이터레이터의 전체 내용을 리스트에 넣는다
    - 메모리를 엄청 많이 사용할 수 있다.
- 호출될 때마다 새로운 이터레이터를 반환하는 함수를 받는다.
    - 보기에 좋지 않다.
- **이터레이터 프로토콜(iterator protocol)**
    - 이터레이터 프로토콜은 파이썬의 `for`루프나 그와 연관된 식들이 컨테이너 타입의 내용을 방문할 때 사용하는 절차이다.

In [9]:
class ReadVisits:
    def __init__(self, data_path):
        self.data_path = data_path
    
    def __iter__(self):
        with open(self.data_path) as f:
            for line in f:
                yield int(line)

In [10]:
visits = ReadVisits('numbers.txt')
percentages = normalize(visits)
print(percentages)
assert sum(percentages) == 100.0

[11.538461538461538, 26.923076923076923, 61.53846153846154]


- 이 코드가 잘 작동하는 이유는 `normalize` 함수 안의 `sum` 메서드가 `ReadVisits.__iter__`를 호출해서 새로운 이터레이터 객체를 할당하기 때문이다.
- 각 숫자를 정규화하기 위한 `for` 루프도 `__iter__`를 호출해서 두 번째 이터레이터 객체를 만든다.

- 두 이터레이터는 서로 독립적으로 진행되고 소진된다. 이로 인해 두 이터레이터는 모든 입력 데이터 값을 볼 수 있는 별도의 이터레이터를 만들어낸다.

### 인자가 이터레이터인지 검사

In [23]:
def normalize_defensive(numbers):
    if iter(numbers) is numbers: # is를 사용해 iterator인지 검사 iter(it) is it
        raise TypeError('컨테이너를 제공해야 합니다')
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

In [20]:
s = 'abc' # iterable
# print(dir(s))
it = iter(s) # iterator
# print(iter(it) is it)
print(dir(it))

True
['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__length_hint__', '__lt__', '__ne__', '__new__', '__next__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__']


`collections.abc` 내장 모듈은 `isinstance`를 사용해 잠재적인 문제를 검사할 수 있는 `Iterator` 클래스를 제공한다.

In [11]:
from collections.abc import Iterator

def normalize_defensive(numbers):
    if isinstance(numbers, Iterator): # 반복 가능한 이터레이터인지 검사하는 다른 방법
        raise TypeError('컨테이너를 제공해야 합니다')
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

In [13]:
visits = [15, 35, 80]
percentages = normalize_defensive(visits)
assert sum(percentages) == 100.0

visits = ReadVisits('numbers.txt')
percentages = normalize_defensive(visits)
assert sum(percentages) == 100.0

#
visits = [15, 35, 80]
it = iter(visits)
# 오류가 나는 부분. 오류를 보고 싶으면 커멘트를 해제할것
# normalize_defensive(it)

- 이 함수는 입력이 컨테이너가 아닌 이터레이터면 예외를 발생시킨다.
- 비동기 이터레이터에 대해서도 같은 접근 방식을 사용할 수 있다.

## BETTER WAY 32 긴 리스트 컴프리헨션보다는 제너레이터 식을 사용하라

파이썬은 **제너레이터 식(generatro expression)**을 제공한다. 제너레이터 식은 리스트 컴프리헨션과 제너레이터를 일반화한 것이다. 제너레이터 식을 실행해도 출력 시퀀스 전체가 실체화되지는 않는다. 그 대신 제너레이터 식에 들어 있는 식으로부터 원소를 하나씩 만들어내는 이터레이터가 생성된다.

`()` 사이에 리스트 컴프리헨션과 비슷한 구문을 넣어 제너레이터 식을 만들 수 있다.

제너레이터 식은 이터레이터로 즉시 평가되며, 더 이상 시퀀스 원소 계산을 하지 않는다.

In [36]:
x = (i for i in range(50) if i % 2 == 0)

print(x)

for i in x:
    print(i, end=" ")

<generator object <genexpr> at 0x7f1da45aadd0>
0 2 4 6 8 10 12 14 16 18 20 22 24 26 28 30 32 34 36 38 40 42 44 46 48 

In [30]:
it = (len(x) for x in open('numbers.txt'))
print(it)

<generator object <genexpr> at 0x7f1da04cbc50>


In [31]:
print(next(it))
print(next(it))

3
3


제터레이터 식의 또 다른 강력한 특징은 두 제너레이터 식을 합성할 수 있다는 점이다. 

다음 코드에서는 제너레티어 식이 반환한 이터레이터를 다른 제너레이터 식의 입력으로 사용된다.

In [33]:
roots = ((x, x**0.5) for x in it) # 이터레이터를 다른 제너레이터 식의 입력으로 사용
print(next(roots))

(2, 1.4142135623730951)


- 제너레이터를 함께 연결한 코드를 파이썬은 아주 빠르게 실행할 수 있다.

- 아주 큰 입력 스트림에 대해 여러 기능을 합성해 적용해야 한다면, 제너레이터 식을 선택하라.

## BETTER WAY 33 yield from을 사용해 여러 제너레이터를 합성하라

반복문을 사용하지 않고 반복 가능한 객체에서 값을 하나씩 바깥으로 전달할 수 있게 해주는 것이 `yield from`이다.

In [34]:
def number_generator():
    x = [1, 2, 3]  # 반복 가능한 객체
    yield from x


for i in number_generator():
    print(i)

1
2
3


`yield from`에 제너레이터 객체를 지정해 줄 수 있다.

In [37]:
def number_generator(stop): # 제너레이터
    n = 0
    while n < stop:
        yield n
        n += 1


def three_generator():
    yield from number_generator(3) # 제너레이터 객체 지정


for i in three_generator():
    print(i)

0
1
2


In [38]:
def move(period, speed):
    for _ in range(period):
        yield speed

def pause(delay):
    for _ in range(delay):
        yield 0

In [40]:
def animate_composed():
    yield from move(4, 5.0)
    yield from pause(3)
    yield from move(2, 3.0)

def render(delta):
    print(f'Delta: {delta:.1f}')
    # 화면에서 이미지 이동시킨다.

def run(func):
    for delta in func():
        render(delta)

run(animate_composed)

Delta: 5.0
Delta: 5.0
Delta: 5.0
Delta: 5.0
Delta: 0.0
Delta: 0.0
Delta: 0.0
Delta: 3.0
Delta: 3.0


`yield from`은 근본적으로 파이썬 인터프리터가 여러분 대신 `for` 루프를 내포시키고 `yield` 식을 처리하도록 만든다. 이로 인해 성능도 더 좋아진다.

`timeit` 내장 모듈을 통해 마이크로 벤치마크(micro benchmark)를 실행함으로써 성능이 개선되는지 살펴본다.

In [1]:
import timeit

def child():
    for i in range(1_000_000):
        yield i

def slow():
    for i in child():
        yield i # yield 사용

def fast():
    yield from child() # yield from 사용

baseline = timeit.timeit(
    stmt='for _ in slow(): pass',
    globals=globals(),
    number=50)
print(f'수동 내포: {baseline:.2f}s')

comparison = timeit.timeit(
    stmt='for _ in fast(): pass',
    globals=globals(),
    number=50)
print(f'합성 사용: {comparison:.2f}s')

reduction = -(comparison - baseline) / baseline
print(f'{reduction:.1%} 시간이 적게 듦')

수동 내포: 4.84s
합성 사용: 4.01s
17.2% 시간이 적게 듦


## BETTER WAY 34 send로 제너레이터에 데이터를 주입하지 말라

In [2]:
import math

# 소프트웨어 라디오를 사용해 신호를 내보낸다.
# 주어진 간격과 진폭에 따른 사인파값을 생성한다.

def wave(amplitude, steps):
    step_size = 2 * math.pi / steps   # 2라디안/단계 수
    for step in range(steps):
        radians = step * step_size
        fraction = math.sin(radians)
        output = amplitude * fraction
        yield output

def transmit(output):
    if output is None:
        print(f'출력: None')
    else:
        print(f'출력: {output:>5.1f}')

def run(it):
    for output in it:
        transmit(output)

run(wave(3.0, 8))

출력:   0.0
출력:   2.1
출력:   3.0
출력:   2.1
출력:   0.0
출력:  -2.1
출력:  -3.0
출력:  -2.1


파이썬 제너레이터는 `send` 메서드를 지원한다. 이 메서등는 `yield` 식을 양방향 채널로 격상시켜준다.

`send` 메서드를 사용하면 입력을 제너레이터에 스트리밍하는 동시에 출력을 내보낼 수 있다. 일반적으로 제너레이터를 이터레이션할 때 `yield`식이 반환하는 값은 `None`이다.

In [3]:
def my_generator():
    received = yield 1
    print(f'받은 값 = {received}')

it = iter(my_generator())
output = next(it)
print(f'출력값 = {output}')

try:
    next(it)
except StopIteration:
    pass


it = iter(my_generator())
output = it.send(None)     # 첫 번째 제너레이터 출력을 얻는다
print(f'출력값 = {output}')

try:
    it.send('안녕!')    # 값을 제너레이터에 넣는다
except StopIteration:
    pass

출력값 = 1
받은 값 = None
출력값 = 1
받은 값 = 안녕!


- `for` 루프나 `next` 내장 함수로 제너레이터를 이터레이션하지 않고 `send` 메서드를 호출하면, 제너레이터가 재개될 때 `yield`가 `send`에 전달된 파라미터 값을 반환한다.


- 하지만 방금 시작한 제너레이터는 아직 `yield` 식에 도달하지 못했기 때문에 최초로 `send`를 호출할 때 인자로 전달할 수 있는 유일한 값은 `None`뿐이다.

In [4]:
import math
def wave_modulating(steps):
    step_size = 2 * math.pi / steps
    amplitude = yield                # 초기 진폭을 받는다
    for step in range(steps):
        radians = step * step_size
        fraction = math.sin(radians)
        output = amplitude * fraction
        amplitude = yield output     # 다음 진폭을 받는다

def run_modulating(it):
    amplitudes = [
        None, 7, 7, 7, 2, 2, 2, 2, 10, 10, 10, 10, 10] # 진폭
    for amplitude in amplitudes:
        output = it.send(amplitude) # amplitudee를 보낸다
        transmit(output)

run_modulating(wave_modulating(12))

출력: None
출력:   0.0
출력:   3.5
출력:   6.1
출력:   2.0
출력:   1.7
출력:   1.0
출력:   0.0
출력:  -5.0
출력:  -8.7
출력: -10.0
출력:  -8.7
출력:  -5.0


- 위의 경우는 너무 복잡하다.
- `yield from`을 사용해보자

In [5]:
def complex_wave():
    yield from wave(7.0, 3)
    yield from wave(2.0, 4)
    yield from wave(10.0, 5)

run(complex_wave())

출력:   0.0
출력:   6.1
출력:  -6.1
출력:   0.0
출력:   2.0
출력:   0.0
출력:  -2.0
출력:   0.0
출력:   9.5
출력:   5.9
출력:  -5.9
출력:  -9.5


In [6]:
def complex_wave_modulating():
    yield from wave_modulating(3)
    yield from wave_modulating(4)
    yield from wave_modulating(5)
    
run_modulating(complex_wave_modulating())

출력: None
출력:   0.0
출력:   6.1
출력:  -6.1
출력: None
출력:   0.0
출력:   2.0
출력:   0.0
출력: -10.0
출력: None
출력:   0.0
출력:   9.5
출력:   5.9


- 출력에 `None`이 여럿 보인다. 
- 내포된 제너레이터에 대한 `yield from` 식이 끝날 때마다 다음 `yield from`식이 실행된다. 
- 각각의 내포된 제너레이터는 `send` 메서드 호출로부터 값을 받기 위해 아무런 값도 만들어내지 않는 단순한 `yield` 식으로 시작한다. 이로 인해 부모 제너레이터가 자식 제너레이터를 옮겨갈 때마다 `None`이 출력된다.

가장 쉬운 해결책은 `send` 메서드를 사용하지 않고 wave 함수에 이터레이터를 전달하는 것이다. 이 이터레인터는 자신에 대해 `next` 내장 함수가 호출될 때마다 입력으로 받은 진폭을 하나씩 돌려준다.

이런 식으로 이전 제너레이터를 다음 제너레이터의 입력으로 연쇄시켜 연결하면 입력과 출력이 차례로 처리되게 만들 수 있다.

In [7]:
import math

def wave_cascading(amplitude_it, steps):
    step_size = 2 * math.pi / steps
    for step in range(steps):
        radians = step * step_size
        fraction = math.sin(radians)
        amplitude = next(amplitude_it) # 다음 입력 받기
        output = amplitude * fraction
        yield output

def complex_wave_cascading(amplitude_it):
    yield from wave_cascading(amplitude_it, 3)
    yield from wave_cascading(amplitude_it, 4)
    yield from wave_cascading(amplitude_it, 5)

def run_cascading():
    amplitudes = [7, 7, 7, 2, 2, 2, 2, 10, 10, 10, 10, 10]
    it = complex_wave_cascading(iter(amplitudes))
    for amplitude in amplitudes:
        output = next(it)
        transmit(output)

run_cascading()

출력:   0.0
출력:   6.1
출력:  -6.1
출력:   0.0
출력:   2.0
출력:   0.0
출력:  -2.0
출력:   0.0
출력:   9.5
출력:   5.9
출력:  -5.9
출력:  -9.5


- 이 접근 방법에서 가장 멋진 부분은 아무데서나 이터레이터를 가져올 수 있다는 점이다. 
- 이터레이터가 완전히 동적인 경우(제너레이터 함수를 사용해 이터레이터를 만든 경우)에도 잘 작동한다.
- 다만 이 코드는 입력 제너레이터가 완전히 스레드 안전하다고 가정하다는 단점이 있다. 제너레이터는 항상 스레드 안전하지는 않다.

## BETTER WAY 35 제너레이터 안에서 throw로 상태를 변화시키지 말라

제너레이터 안에는 `Exception`을 다시 던질 수 있는 `throw` 메서드가 있다.

어떤 제너레이터에 대해 `throw`가 호출되면 이 제너레이터는 값을 내놓는 `yield`로부터 평소처럼 제너레이터 실행을 계속 하는 대신, `throw`가 제공한 `Exception`을 다시 던진다.

In [9]:
class MyError(Exception):
    pass

def my_generator():
    yield 1
    yield 2
    yield 3

it = my_generator()
print(next(it))  # 1을 내놓음
print(next(it))  # 2를 내놓음
#print(it.throw(MyError('test error'))) # 에러 발생 시킨다

1
2


In [14]:
def my_generator():
    yield 1
    try:
        yield 2
    except MyError:
        print('MyError 발생!')
    else: # else
        yield 3
    yield 4

it = my_generator()
print(next(it))  # 1을 내놓음
print(next(it))  # 2를 내놓음
print(it.throw(MyError('test error')))


1
2
MyError 발생!
4


- 이 기능은 제너레이터와 제너레이터를 호출하는 쪽 사이에 양방향 통신 수단을 제공한다. 경우에 따라 이 양방향 통신 수단이 유용할 수도 있다.

In [29]:
class Reset(Exception):
    pass

# 타이머 함수
def timer(period):
    current = period
    while current:
        current -= 1
        try:
            yield current
        except Reset: # 예외 발생하면 시간 리셋
            current = period

RESETS = [
    False, False, False, True, False, True, False,
    False, False, False, False, False, False, False]

def check_for_reset():
    # 외부 이벤트를 폴링한다
    return RESETS.pop(0)

def announce(remaining):
    print(f'{remaining} 틱 남음')

def run():
    it = timer(4)
    while True:
        try:
            if check_for_reset():
                current = it.throw(Reset()) # throw로 상태 변화
            else:
                current = next(it)
        except StopIteration:
            break
        else:
            announce(current)

run()

3 틱 남음
2 틱 남음
1 틱 남음
3 틱 남음
2 틱 남음
3 틱 남음
2 틱 남음
1 틱 남음
0 틱 남음


- 이 코드는 동작은 잘하지만 필요 이상으로 읽기 어렵다.


In [30]:
class Timer:
    def __init__(self, period):
        self.current = period
        self.period = period

    def reset(self):
        self.current = self.period

    def __iter__(self):
        while self.current:
            self.current -= 1
            yield self.current


def run():
    timer = Timer(3)
    for current in timer:
        if check_for_reset():
            timer.reset()
        announce(current)

run()

2 틱 남음
1 틱 남음
0 틱 남음


## 38번 하면 돼